In [4]:
# main.py
import csv
import json
from typing import List, Optional, Tuple

from fastapi import (
    FastAPI,
    Depends,
    HTTPException,
    status,
    APIRouter,
    Header,
    BackgroundTasks,
)
from fastapi.encoders import jsonable_encoder
from pydantic import BaseModel, Field, ConfigDict
from sqlalchemy import create_engine, Column, Integer, String, Float, select, func
from sqlalchemy.orm import declarative_base, Session, sessionmaker

# ---------- БД и ORM ----------

DATABASE_URL = "sqlite:///students.db"
engine = create_engine(DATABASE_URL, future=True, echo=False)
SessionLocal = sessionmaker(bind=engine, autoflush=False, autocommit=False, future=True)

Base = declarative_base()


class Student(Base):
    __tablename__ = "students"

    id = Column(Integer, primary_key=True, autoincrement=True)
    last_name = Column(String, nullable=False)
    first_name = Column(String, nullable=False)
    faculty = Column(String, nullable=False)
    course = Column(String, nullable=False)
    grade = Column(Float, nullable=False)


class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, autoincrement=True)
    username = Column(String, unique=True, nullable=False)
    password = Column(String, nullable=False)  # в учебных целях без хеширования


Base.metadata.create_all(engine)

# ---------- Pydantic-схемы ----------


class StudentBase(BaseModel):
    last_name: str = Field(..., json_schema_extra={"example": "Иванов"})
    first_name: str = Field(..., json_schema_extra={"example": "Иван"})
    faculty: str = Field(..., json_schema_extra={"example": "ФПМИ"})
    course: str = Field(..., json_schema_extra={"example": "Мат. Анализ"})
    grade: float = Field(..., ge=0, le=100, json_schema_extra={"example": 72})


class StudentCreate(StudentBase):
    pass


class StudentUpdate(BaseModel):
    last_name: Optional[str] = Field(None, json_schema_extra={"example": "Петров"})
    first_name: Optional[str] = Field(None, json_schema_extra={"example": "Пётр"})
    faculty: Optional[str] = Field(None, json_schema_extra={"example": "РЭФ"})
    course: Optional[str] = Field(None, json_schema_extra={"example": "Информатика"})
    grade: Optional[float] = Field(None, ge=0, le=100, json_schema_extra={"example": 88})


class StudentOut(StudentBase):
    id: int
    model_config = ConfigDict(from_attributes=True)


class UserBase(BaseModel):
    username: str = Field(..., json_schema_extra={"example": "user1"})


class UserCreate(UserBase):
    password: str = Field(..., min_length=4, json_schema_extra={"example": "secret"})


class UserLogin(UserBase):
    password: str = Field(..., min_length=4, json_schema_extra={"example": "secret"})


class UserOut(UserBase):
    id: int
    model_config = ConfigDict(from_attributes=True)


class AvgOut(BaseModel):
    faculty: str
    average_grade: float


class DeleteRequest(BaseModel):
    ids: List[int]


# ---------- DI: сессия и текущий пользователь ----------


def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


def get_current_user(
    x_user_id: int = Header(..., alias="X-User-Id"),
    db: Session = Depends(get_db),
) -> User:
    user = db.get(User, x_user_id)
    if not user:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Неверный X-User-Id или пользователь не существует",
        )
    return user


# ---------- Redis-кэш (с graceful fallback) ----------

try:
    import redis  # type: ignore

    redis_client = redis.Redis(
        host="localhost", port=6379, db=0, decode_responses=True
    )
except Exception:  # нет пакета или нет сервера
    redis_client = None

CACHE_TTL_SECONDS = 60


def make_cache_key(prefix: str, **kwargs) -> str:
    parts = [prefix] + [f"{k}={v}" for k, v in sorted(kwargs.items())]
    return "|".join(parts)


def cache_get(key: str):
    if redis_client is None:
        return None
    try:
        raw = redis_client.get(key)
    except Exception:
        return None
    if raw is None:
        return None
    try:
        return json.loads(raw)
    except json.JSONDecodeError:
        return None


def cache_set(key: str, data, ttl: int = CACHE_TTL_SECONDS):
    if redis_client is None:
        return
    try:
        payload = json.dumps(jsonable_encoder(data))
        redis_client.setex(key, ttl, payload)
    except Exception:
        pass


def clear_cache():
    if redis_client is None:
        return
    try:
        redis_client.flushdb()
    except Exception:
        pass


# ---------- Репозиторий студентов ----------


class StudentRepo:
    def __init__(self, db: Session):
        self.db = db

    # Create
    def create(self, data: StudentCreate) -> Student:
        obj = Student(**data.model_dump())
        self.db.add(obj)
        self.db.commit()
        self.db.refresh(obj)
        clear_cache()
        return obj

    # Read
    def get(self, student_id: int) -> Student:
        obj = self.db.get(Student, student_id)
        if not obj:
            raise HTTPException(status_code=404, detail="Студент не найден")
        return obj

    def list(self, limit: int = 100, offset: int = 0) -> List[Student]:
        stmt = select(Student).offset(offset).limit(limit)
        return list(self.db.execute(stmt).scalars())

    # Update (partial)
    def update(self, student_id: int, data: StudentUpdate) -> Student:
        obj = self.get(student_id)
        for k, v in data.model_dump(exclude_unset=True).items():
            setattr(obj, k, v)
        self.db.commit()
        self.db.refresh(obj)
        clear_cache()
        return obj

    # Delete
    def delete(self, student_id: int) -> None:
        obj = self.get(student_id)
        self.db.delete(obj)
        self.db.commit()
        clear_cache()

    # ---- Запросы из задания ----
    def by_faculty(self, faculty: str) -> List[Student]:
        stmt = (
            select(Student)
            .where(Student.faculty == faculty)
            .order_by(Student.last_name, Student.first_name)
        )
        return list(self.db.execute(stmt).scalars())

    def unique_courses(self) -> List[str]:
        stmt = select(func.distinct(Student.course)).order_by(Student.course)
        return [row[0] for row in self.db.execute(stmt).all()]

    def avg_grade_by_faculty(self) -> List[Tuple[str, float]]:
        stmt = (
            select(Student.faculty, func.avg(Student.grade))
            .group_by(Student.faculty)
            .order_by(Student.faculty)
        )
        return [(fac, float(avg)) for fac, avg in self.db.execute(stmt).all()]

    def below_grade_in_course(self, course: str, threshold: float = 30.0) -> List[Student]:
        stmt = (
            select(Student)
            .where((Student.course == course) & (Student.grade < threshold))
            .order_by(Student.grade.asc())
        )
        return list(self.db.execute(stmt).scalars())

    # Утилита: загрузка CSV
    def load_csv(self, csv_path: str) -> int:
        added = 0
        with open(csv_path, "r", encoding="utf-8-sig", newline="") as f:
            reader = csv.DictReader(f)
            required = ["Фамилия", "Имя", "Факультет", "Курс", "Оценка"]
            missing = [c for c in required if c not in (reader.fieldnames or [])]
            if missing:
                raise HTTPException(
                    status_code=400, detail=f"В CSV нет колонок: {missing}"
                )
            for row in reader:
                self.db.add(
                    Student(
                        last_name=row["Фамилия"].strip(),
                        first_name=row["Имя"].strip(),
                        faculty=row["Факультет"].strip(),
                        course=row["Курс"].strip(),
                        grade=float(row["Оценка"]),
                    )
                )
                added += 1
            self.db.commit()
        clear_cache()
        return added


# ---------- фоновые задачи ----------


def background_load_csv(path: str):
    db = SessionLocal()
    try:
        StudentRepo(db).load_csv(path)
    finally:
        db.close()


def background_delete_students(ids: List[int]):
    db = SessionLocal()
    try:
        repo = StudentRepo(db)
        for sid in ids:
            try:
                repo.delete(sid)
            except HTTPException:
                # если какого-то id нет — просто пропускаем
                pass
    finally:
        db.close()


# ---------- FastAPI и роутеры ----------

app = FastAPI(title="Students CRUD (Auth + BG + Redis)")


auth_router = APIRouter(prefix="/auth", tags=["auth"])


@auth_router.post(
    "/register", response_model=UserOut, status_code=status.HTTP_201_CREATED
)
def register_user(payload: UserCreate, db: Session = Depends(get_db)):
    existing = (
        db.execute(select(User).where(User.username == payload.username))
        .scalar_one_or_none()
    )
    if existing:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Пользователь с таким username уже существует",
        )
    user = User(username=payload.username, password=payload.password)
    db.add(user)
    db.commit()
    db.refresh(user)
    return user


@auth_router.post("/login")
def login_user(payload: UserLogin, db: Session = Depends(get_db)):
    user = (
        db.execute(select(User).where(User.username == payload.username))
        .scalar_one_or_none()
    )
    if not user or user.password != payload.password:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Неверный логин или пароль",
        )
    return {"user_id": user.id}


@auth_router.post("/logout")
def logout_user(current_user: User = Depends(get_current_user)):
    return {"detail": f"Пользователь {current_user.username} разлогинен"}


app.include_router(auth_router)

# ---------- CRUD по студентам + кэш ----------


@app.post("/students", response_model=StudentOut, status_code=status.HTTP_201_CREATED)
def create_student(
    payload: StudentCreate,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    return StudentRepo(db).create(payload)


@app.get("/students/{student_id}", response_model=StudentOut)
def get_student(
    student_id: int,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    key = make_cache_key("student", id=student_id)
    cached = cache_get(key)
    if cached is not None:
        return cached

    obj = StudentRepo(db).get(student_id)
    cache_set(key, obj)
    return obj


@app.get("/students", response_model=List[StudentOut])
def list_students(
    limit: int = 100,
    offset: int = 0,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    key = make_cache_key("students_list", limit=limit, offset=offset)
    cached = cache_get(key)
    if cached is not None:
        return cached

    data = StudentRepo(db).list(limit=limit, offset=offset)
    cache_set(key, data)
    return data


@app.put("/students/{student_id}", response_model=StudentOut)
def update_student(
    student_id: int,
    payload: StudentUpdate,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    return StudentRepo(db).update(student_id, payload)


@app.delete("/students/{student_id}", status_code=status.HTTP_204_NO_CONTENT)
def delete_student(
    student_id: int,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    StudentRepo(db).delete(student_id)
    return


@app.get("/students/by-faculty/{faculty}", response_model=List[StudentOut])
def students_by_faculty(
    faculty: str,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    key = make_cache_key("by_faculty", faculty=faculty)
    cached = cache_get(key)
    if cached is not None:
        return cached

    data = StudentRepo(db).by_faculty(faculty)
    cache_set(key, data)
    return data


@app.get("/courses/unique", response_model=List[str])
def get_unique_courses(
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    key = make_cache_key("unique_courses")
    cached = cache_get(key)
    if cached is not None:
        return cached

    data = StudentRepo(db).unique_courses()
    cache_set(key, data)
    return data


@app.get("/faculties/avg-grade", response_model=List[AvgOut])
def avg_grade_by_faculty(
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    key = make_cache_key("avg_grade_by_faculty")
    cached = cache_get(key)
    if cached is not None:
        return cached

    data = StudentRepo(db).avg_grade_by_faculty()
    result = [{"faculty": fac, "average_grade": avg} for fac, avg in data]
    cache_set(key, result)
    return result


@app.get("/students/by-course/below", response_model=List[StudentOut])
def below_threshold(
    course: str,
    threshold: float = 30.0,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    key = make_cache_key("below_threshold", course=course, threshold=threshold)
    cached = cache_get(key)
    if cached is not None:
        return cached

    data = StudentRepo(db).below_grade_in_course(course, threshold)
    cache_set(key, data)
    return data


# ---------- фоновые эндпоинты ----------


@app.post("/load-csv")
def load_csv_background(
    path: str,
    background_tasks: BackgroundTasks,
    current_user: User = Depends(get_current_user),
):
    """
    Фоновая задача: наполнение БД данными из CSV.
    """
    background_tasks.add_task(background_load_csv, path)
    return {"detail": f"Задача на загрузку CSV из {path} запущена в фоне"}


@app.post("/students/delete-bulk")
def delete_students_background(
    payload: DeleteRequest,
    background_tasks: BackgroundTasks,
    current_user: User = Depends(get_current_user),
):
    """
    Фоновая задача: удаление студентов по списку id.
    """
    background_tasks.add_task(background_delete_students, payload.ids)
    return {
        "detail": f"Задача на удаление {len(payload.ids)} студентов запущена в фоне"
    }


In [6]:
# test_main.py
import pytest
from fastapi.testclient import TestClient

from __main__ import app, Base, engine

client = TestClient(app)


# --- фикстура: перед КАЖДЫМ тестом пересоздаём таблицы ---
@pytest.fixture(autouse=True)
def reset_db():
    Base.metadata.drop_all(bind=engine)
    Base.metadata.create_all(bind=engine)
    yield


# --- вспомогалки ---


def register_user(username="user1", password="secret"):
    resp = client.post(
        "/auth/register",
        json={"username": username, "password": password},
    )
    return resp


def auth_headers(username="user1", password="secret"):
    """Регистрирует пользователя и отдаёт заголовок X-User-Id."""
    resp = register_user(username, password)
    assert resp.status_code == 201
    user_id = resp.json()["id"]
    return {"X-User-Id": str(user_id)}


# ---------------------------------------------------------
# 1) /auth/register
# ---------------------------------------------------------


def test_register_success():
    resp = register_user("alice", "secret")
    assert resp.status_code == 201
    data = resp.json()
    assert "id" in data
    assert data["username"] == "alice"


def test_register_duplicate_username():
    # первый раз ок
    resp1 = register_user("alice", "secret")
    assert resp1.status_code == 201

    # второй раз — ошибка 400
    resp2 = register_user("alice", "secret")
    assert resp2.status_code == 400
    assert "detail" in resp2.json()


# ---------------------------------------------------------
# 2) /auth/login
# ---------------------------------------------------------


def test_login_success():
    register_user("bob", "secret")
    resp = client.post(
        "/auth/login",
        json={"username": "bob", "password": "secret"},
    )
    assert resp.status_code == 200
    data = resp.json()
    assert "user_id" in data
    assert isinstance(data["user_id"], int)


def test_login_wrong_password():
    register_user("bob", "secret")
    resp = client.post(
        "/auth/login",
        json={"username": "bob", "password": "wrong"},
    )
    assert resp.status_code == 401
    assert resp.json()["detail"] == "Неверный логин или пароль"


# ---------------------------------------------------------
# 3) POST /students
# ---------------------------------------------------------


def test_create_student_success():
    headers = auth_headers()

    payload = {
        "last_name": "Иванов",
        "first_name": "Иван",
        "faculty": "ФПМИ",
        "course": "Мат. Анализ",
        "grade": 72,
    }

    resp = client.post("/students", json=payload, headers=headers)
    assert resp.status_code == 201
    data = resp.json()
    assert data["last_name"] == "Иванов"
    assert data["faculty"] == "ФПМИ"
    assert "id" in data


def test_create_student_invalid_user():
    # пользователь с id 999 не существует
    headers = {"X-User-Id": "999"}

    payload = {
        "last_name": "Петров",
        "first_name": "Пётр",
        "faculty": "ФПМИ",
        "course": "Информатика",
        "grade": 80,
    }

    resp = client.post("/students", json=payload, headers=headers)
    assert resp.status_code == 401
    assert "detail" in resp.json()


# ---------------------------------------------------------
# 4) GET /students/{student_id}
# ---------------------------------------------------------


def test_get_student_success():
    headers = auth_headers()

    # сначала создаём студента
    create_resp = client.post(
        "/students",
        json={
            "last_name": "Иванов",
            "first_name": "Иван",
            "faculty": "ФПМИ",
            "course": "Мат. Анализ",
            "grade": 72,
        },
        headers=headers,
    )
    assert create_resp.status_code == 201
    student_id = create_resp.json()["id"]

    # теперь читаем
    get_resp = client.get(f"/students/{student_id}", headers=headers)
    assert get_resp.status_code == 200
    data = get_resp.json()
    assert data["id"] == student_id
    assert data["last_name"] == "Иванов"


def test_get_student_not_found():
    headers = auth_headers()
    resp = client.get("/students/999", headers=headers)
    assert resp.status_code == 404
    assert resp.json()["detail"] == "Студент не найден"


# ---------------------------------------------------------
# 5) GET /students/by-faculty/{faculty}
# ---------------------------------------------------------


def test_students_by_faculty_success():
    headers = auth_headers()

    # два студента ФПМИ
    for name in ["Иванов", "Петров"]:
        client.post(
            "/students",
            json={
                "last_name": name,
                "first_name": "Иван",
                "faculty": "ФПМИ",
                "course": "Мат. Анализ",
                "grade": 70,
            },
            headers=headers,
        )

    # один студент с другим факультетом
    client.post(
        "/students",
        json={
            "last_name": "Сидоров",
            "first_name": "Иван",
            "faculty": "РЭФ",
            "course": "Информатика",
            "grade": 90,
        },
        headers=headers,
    )

    resp = client.get("/students/by-faculty/ФПМИ", headers=headers)
    assert resp.status_code == 200
    data = resp.json()
    assert len(data) == 2
    assert all(st["faculty"] == "ФПМИ" for st in data)


def test_students_by_faculty_invalid_auth():
    # без корректного пользователя (невалидный id)
    headers = {"X-User-Id": "999"}
    resp = client.get("/students/by-faculty/ФПМИ", headers=headers)
    assert resp.status_code == 401
    assert "detail" in resp.json()
